# Lecture 18 : GPU Extreme

# **Ensure that Runtime is a CPU**

## Clone the materials repo to access datafiles.

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 20.01 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

# Part 1 : Sequential Extreme

In [3]:
%%writefile extreme.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>

typedef unsigned char byte;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

typedef struct {
    int max_dist_sq;
    int i,j;
} extreme_info;

int vec_dist_sq(byte* u, byte* v, int dim) {
    int dist_sq = 0;
    for (int i=0;i<dim;i++) {
	    dist_sq += (u[i]-v[i])*(u[i]-v[i]);
    }
    return dist_sq;
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 60000;
    int dim = 784;
    byte* data = (byte*)malloc(len*dim*sizeof(byte));
    char images_file[] = "train-images-idx3-ubyte";
    read_bin(data,len*dim,images_file,16);

    // start the timer
    clock_t start = clock();

    // find the extreme pair
    extreme_info info = { 0, -1, -1 };
    for (int i=0;i<len-1;i++) {
	    for (int j=i+1;j<len;j++) {
	        int dist_sq = vec_dist_sq(data+i*dim,data+j*dim,dim);
	        if (dist_sq > info.max_dist_sq) {
		        info.max_dist_sq = dist_sq;
		        info.i = i;
		        info.j = j;
	        }
	    }
    }

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(info.max_dist_sq));
    printf ("extreme pair = (%d,%d)\n",info.i,info.j);

    // free dynamically allocated memory
    free(data);
}


Writing extreme.c


In [4]:
!gcc -O3 -o extreme extreme.c -lm

In [5]:
!./extreme

number of images = 60000
elapsed time = 209.5897 seconds
extreme distance = 4303.32
extreme pair = (26785,59452)


## Note that we are reading in the binary version of the MNIST image file rather than the text version.

## Binary files take up less space than text files and they load faster as well.

# Part 2 : GPU Extreme (Max Distance Only)

## We start by writing a version that just computes the maximum distance squared over all pairs.  

## Note that each thread computes the distance squared between a single pair of points.

## Also note that a thread only considers its assigned pair if i < j.  

## Since the *vec_dist_sq* function runs on the device we have to declare the function as:

    __device__ int vec_dist_sq(byte* u, byte* v, int dim) {

## Note the use of *atomicMax* to keep track of max_dist_sq.

## When working with data on a GPU it is easy to run out of device memory.  

##Thus, to be safe it is best to check the return value of cudaMalloc for NULL.

# **Switch runtime to a T4**

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 7.66 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

In [3]:
%%writefile gpu_extreme_v1.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

typedef unsigned char byte;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

__device__ int vec_dist_sq(byte* u, byte* v, int dim) {
    int dist_sq = 0;
    for (int i=0;i<dim;i++) {
	    dist_sq += (u[i]-v[i])*(u[i]-v[i]);
    }
    return dist_sq;
}

__global__ void extremeKernel(byte* data, int len, int dim, int* max_dist_sq) {
    int thread_num = blockIdx.x*blockDim.x + threadIdx.x;
    if (thread_num < len*len) {
	    int i = thread_num/len;
	    int j = thread_num%len;
	    if (i < j) {
	        int dist_sq = vec_dist_sq(data+i*dim,data+j*dim,dim);
	        atomicMax(max_dist_sq,dist_sq);
	    }
    }
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 10000;
    int dim = 784;
    byte* data = (byte*)malloc(len*dim*sizeof(byte));
    char images_file[] = "t10k-images-idx3-ubyte";
    read_bin(data,len*dim,images_file,16);

    // allocate device memory
    byte* d_data;
    int* d_max_dist_sq;
    cudaMalloc(&d_data,len*dim*sizeof(byte));
    if (d_data == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_max_dist_sq,sizeof(int));
    if (d_max_dist_sq == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }

    // start the timer
    clock_t start = clock();

    // copy data to device
    cudaMemcpy(d_data,data,len*dim*sizeof(byte),cudaMemcpyHostToDevice);

    // initialize the device max_dist_sq to 0
    cudaMemset(d_max_dist_sq,0,sizeof(int));

    // launch kernel to compute extreme distance
    int B = 256;
    int G = (len*len+B-1)/B;
    extremeKernel <<< G, B >>> (d_data,len,dim,d_max_dist_sq);

    // copy max_dist_sq from device to host
    int max_dist_sq;
    cudaMemcpy(&max_dist_sq,d_max_dist_sq,sizeof(int),cudaMemcpyDeviceToHost);

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(max_dist_sq));

    // free dynamically allocated memory
    free(data);
    cudaFree(d_data);
    cudaFree(d_max_dist_sq);
}


Writing gpu_extreme_v1.cu


In [4]:
!nvcc -arch=sm_75 -o gpu_extreme_v1 gpu_extreme_v1.cu

In [5]:
!./gpu_extreme_v1

number of images = 10000
elapsed time = 0.8067 seconds
extreme distance = 4097.95


# **Switch runtime to a CPU**

# Part 3 : GPU Extreme (Max Distance and Extreme Pair)

## In CUDA there is no equivalent to an OpenMP critical region.

## Thus, we frequently have to get creative to get the most out of the GPU atomics.

## In order to do an *atomic update* of the triple

$$(max\_dist\_sq, i, j)$$

## we pack the three values $max\_dist\_sq$, $i$, and $j$ into an *unsigned long long* which is 64 bits.  

## We use 32 of the 64 bits to store $\max\_dist\_sq$ and 16 bits each to store $i$ and $j$.

## We put $max\_dist\_sq$ in the high 32 bits so that the *atomicMax* will still work as expected.  

## The extreme pair $(i,j)$ is tucked into the low 32 bits and will not effect the *atomicMax* calculation (unless there is a tie).

## Note that there is a function to *compress* a triple of extreme info and a function to *expand* an unsigned long long into a triple of extreme info.  

## Since the *compress* function runs on the device we have to declare the function as:

    __device__ uint64 extreme_info_compress(int dist_sq, int i, int j) {

# **Switch runtime to a T4**

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 7.72 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

In [3]:
%%writefile gpu_extreme_v2.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

typedef unsigned char byte;
typedef unsigned long long uint64;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

__device__ int vec_dist_sq(byte* u, byte* v, int dim) {
    int dist_sq = 0;
    for (int i=0;i<dim;i++) {
	    dist_sq += (u[i]-v[i])*(u[i]-v[i]);
    }
    return dist_sq;
}

void extreme_info_expand(uint64 info, int* dist_sq, int* i, int* j) {
    *j = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *i = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *dist_sq = info;
}

__device__ uint64 extreme_info_compress(int dist_sq, int i, int j) {
    uint64 info = dist_sq;
    info = info << 16; // shift 16 bits to the left
    info += i;
    info = info << 16; // shift 16 bits to the left
    info += j;
    return info;
}

__global__ void extremeKernel(byte* data, int len, int dim, uint64* max_info) {
    int thread_num = blockIdx.x*blockDim.x + threadIdx.x;
    if (thread_num < len*len) {
	    int i = thread_num/len;
	    int j = thread_num%len;
	    if (i < j) {
	        int dist_sq = vec_dist_sq(data+i*dim,data+j*dim,dim);
	        uint64 info = extreme_info_compress(dist_sq,i,j);
	        atomicMax(max_info,info);
	    }
    }
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 10000;
    int dim = 784;
    byte* data = (byte*)malloc(len*dim*sizeof(byte));
    char images_file[] = "t10k-images-idx3-ubyte";
    read_bin(data,len*dim,images_file,16);

    // allocate device memory
    byte* d_data;
    uint64* d_max_info;
    cudaMalloc(&d_data,len*dim*sizeof(byte));
    if (d_data == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_max_info,sizeof(uint64));
    if (d_max_info == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }

    // start the timer
    clock_t start = clock();

    // copy data to device
    cudaMemcpy(d_data,data,len*dim*sizeof(byte),cudaMemcpyHostToDevice);

    // initialize the device max_info to 0
    cudaMemset(d_max_info,0,sizeof(uint64));

    // launch kernel to compute extreme distance
    int B = 256;
    int G = (len*len+B-1)/B;
    extremeKernel <<< G, B >>> (d_data,len,dim,d_max_info);

    // copy max_info from device to host
    uint64 max_info;
    cudaMemcpy(&max_info,d_max_info,sizeof(uint64),cudaMemcpyDeviceToHost);

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // expand max_info
    int max_dist_sq, i, j;
    extreme_info_expand(max_info,&max_dist_sq,&i,&j);

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(max_dist_sq));
    printf ("extreme pair = (%d,%d)\n",i,j);

    // free dynamically allocated memory
    free(data);
    cudaFree(d_data);
    cudaFree(d_max_info);
}


Writing gpu_extreme_v2.cu


In [4]:
!nvcc -arch=sm_75 -o gpu_extreme_v2 gpu_extreme_v2.cu

In [5]:
!./gpu_extreme_v2

number of images = 10000
elapsed time = 0.7960 seconds
extreme distance = 4097.95
extreme pair = (5977,6412)


# **Switch runtime to a CPU**

# Part 4 : GPU Extreme (Column Major Order)

## It is frequently better to use matrices stored in column major order in CUDA.

## Ideally, consecutive threads in a warp will read consecutive memory locations when accessing memory.  

## Suppose each thread in a warp is reading consecutive rows of a matrix.  

## If the matrix is stored in *row major order* then consecutive threads in a warp are reading values that are stored far apart in memory.  

## However if the matrix is stored in *column major order* then consecutive threads in a warp are reading values that are stored next to each other in memory.

## The main difference to the code is where we calculate the distance between two vectors.

## Here is the distance calculation when storing the data in **row major order**.

    int dist_sq = 0;
	for (int k=0;k<dim;k++) {
		int diff = data[i*dim+k]-data[j*dim+k];
		dist_sq += diff*diff;
	}

## Here is the distance calculation when storing the data in **column major order**.

    int dist_sq = 0;
	for (int k=0;k<dim;k++) {
        int diff = data[k*len+i]-data[k*len+j];
		dist_sq += diff*diff;
	}

# **Switch runtime to a T4**

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

In [3]:
%%writefile gpu_extreme_v3.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

typedef unsigned char byte;
typedef unsigned long long uint64;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

void extreme_info_expand(uint64 info, int* dist_sq, int* i, int* j) {
    *j = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *i = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *dist_sq = info;
}

__device__ uint64 extreme_info_compress(int dist_sq, int i, int j) {
    uint64 info = dist_sq;
    info = info << 16; // shift 16 bits to the left
    info += i;
    info = info << 16; // shift 16 bits to the left
    info += j;
    return info;
}

__global__ void extremeKernel(byte* data, int len, int dim, uint64* max_info) {
    int thread_num = blockIdx.x*blockDim.x + threadIdx.x;
    if (thread_num < len*len) {
	    int i = thread_num/len;
	    int j = thread_num%len;
	    if (i < j) {
	        int dist_sq = 0;
	        for (int k=0;k<dim;k++) {
                int diff = data[k*len+i]-data[k*len+j];
		        dist_sq += diff*diff;
	        }
	        uint64 info = extreme_info_compress(dist_sq,i,j);
	        atomicMax(max_info,info);
	    }
    }
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 10000;
    int dim = 784;
    byte* data = (byte*)malloc(len*dim*sizeof(byte));
    // read the file that thas the dataset stored in column major order
    char images_file[] = "t10k-images-idx3-ubyte-c";
    read_bin(data,len*dim,images_file,16);

    // allocate device memory
    byte* d_data;
    uint64* d_max_info;
    cudaMalloc(&d_data,len*dim*sizeof(byte));
    if (d_data == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_max_info,sizeof(uint64));
    if (d_max_info == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }

    // start the timer
    clock_t start = clock();

    // copy data to device
    cudaMemcpy(d_data,data,len*dim*sizeof(byte),cudaMemcpyHostToDevice);

    // initialize the device max_info to 0
    cudaMemset(d_max_info,0,sizeof(uint64));

    // launch kernel to compute extreme distance
    int B = 256;
    int G = (len*len+B-1)/B;
    extremeKernel <<< G, B >>> (d_data,len,dim,d_max_info);

    // copy max_info from device to host
    uint64 max_info;
    cudaMemcpy(&max_info,d_max_info,sizeof(uint64),cudaMemcpyDeviceToHost);

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // expand max_info
    int max_dist_sq, i, j;
    extreme_info_expand(max_info,&max_dist_sq,&i,&j);

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(max_dist_sq));
    printf ("extreme pair = (%d,%d)\n",i,j);

    // free dynamically allocated memory
    free(data);
    cudaFree(d_data);
    cudaFree(d_max_info);
}


Writing gpu_extreme_v3.cu


In [4]:
!nvcc -arch=sm_75 -o gpu_extreme_v3 gpu_extreme_v3.cu

In [11]:
!./gpu_extreme_v3

number of images = 10000
elapsed time = 0.2508 seconds
extreme distance = 4097.95
extreme pair = (5977,6412)


# **Switch runtime to a CPU**

# Part 5 : GPU Extreme (60000 images)

## When running on a file with 60000 images, there are 3.6 billion threads which is a number that is too large to store in a C int.  

## Thus we have to change some parts of the code to avoid overflow.  

## Here are the necessary changes:

    long long thread_num = (long long)blockIdx.x*blockDim.x + threadIdx.x;

    if (thread_num < (long long)len*len) {

    int G = ((long long)len*len+B-1)/B;

# **Switch runtime to a T4**

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 7.69 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

In [3]:
%%writefile gpu_extreme_v4.cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <cuda.h>

typedef unsigned char byte;
typedef unsigned long long uint64;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

void extreme_info_expand(uint64 info, int* dist_sq, int* i, int* j) {
    *j = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *i = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *dist_sq = info;
}

__device__ uint64 extreme_info_compress(int dist_sq, int i, int j) {
    uint64 info = dist_sq;
    info = info << 16; // shift 16 bits to the left
    info += i;
    info = info << 16; // shift 16 bits to the left
    info += j;
    return info;
}

__global__ void extremeKernel(byte* data, int len, int dim, uint64* max_info) {
    long long thread_num = (long long)blockIdx.x*blockDim.x + threadIdx.x;
    if (thread_num < (long long)len*len) {
	    int i = thread_num/len;
	    int j = thread_num%len;
	    if (i < j) {
	        int dist_sq = 0;
	        for (int k=0;k<dim;k++) {
                int diff = data[k*len+i]-data[k*len+j];
		        dist_sq += diff*diff;
	        }
	        uint64 info = extreme_info_compress(dist_sq,i,j);
	        atomicMax(max_info,info);
	    }
    }
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 60000;
    int dim = 784;
    byte* data = (byte*)malloc(len*dim*sizeof(byte));
    // read the file that thas the dataset stored in column major order
    char images_file[] = "train-images-idx3-ubyte-c";
    read_bin(data,len*dim,images_file,16);

    // allocate device memory
    byte* d_data;
    uint64* d_max_info;
    cudaMalloc(&d_data,len*dim*sizeof(byte));
    if (d_data == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_max_info,sizeof(uint64));
    if (d_max_info == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }

    // start the timer
    clock_t start = clock();

    // copy data to device
    cudaMemcpy(d_data,data,len*dim*sizeof(byte),cudaMemcpyHostToDevice);

    // initialize the device max_info to 0
    cudaMemset(d_max_info,0,sizeof(uint64));

    // launch kernel to compute extreme distance
    int B = 256;
    int G = ((long long)len*len+B-1)/B;
    extremeKernel <<< G, B >>> (d_data,len,dim,d_max_info);

    // copy max_info from device to host
    uint64 max_info;
    cudaMemcpy(&max_info,d_max_info,sizeof(uint64),cudaMemcpyDeviceToHost);

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // expand max_info
    int max_dist_sq, i, j;
    extreme_info_expand(max_info,&max_dist_sq,&i,&j);

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(max_dist_sq));
    printf ("extreme pair = (%d,%d)\n",i,j);

    // free dynamically allocated memory
    free(data);
    cudaFree(d_data);
    cudaFree(d_max_info);
}


Writing gpu_extreme_v4.cu


In [4]:
!nvcc -arch=sm_75 -o gpu_extreme_v4 gpu_extreme_v4.cu

In [5]:
!./gpu_extreme_v4

number of images = 60000
elapsed time = 12.3451 seconds
extreme distance = 4303.32
extreme pair = (26785,59452)


# **Switch runtime to a CPU**

# Part 6 : GPU Extreme (with cuBLAS)

## Although the GPU version 4 is considerably faster than the sequential code, we are not even close to realizing the maximum performance of the GPU for this problem.  

## In the final version we will improve the performance drastically by leveraging the fact that GPUs are extremely fast at matrix multiplication when using special hardware called **Tensor Cores**.

## Suppose we have *n* data points (e.g. images)
## $$\text{data }= \{ \mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_n \}$$
## To find the extreme pair we need to compute roughly $O(n^2)$ squared distances:
## $$\| \mathbf{v}_i - \mathbf{v}_j \|^2 =
(\mathbf{v}_i - \mathbf{v}_j) \cdot
(\mathbf{v}_i - \mathbf{v}_j) =
\mathbf{v}_i \cdot \mathbf{v}_i - 2 \mathbf{v}_i \cdot \mathbf{v}_j + \mathbf{v}_j \cdot \mathbf{v}_j$$

## The majority of the work to be done is computing the $n^2$ dot products
## $$\mathbf{v}_i \cdot \mathbf{v}_j$$
## The key observation is that we can turn computing those dot products into matrix multiplication.  
## $$\begin{bmatrix} \mathbf{v}_1^T \\ \mathbf{v}_2^T \\ \vdots \\ \mathbf{v}_n^T \end{bmatrix}
\begin{bmatrix} \mathbf{v}_1 & \mathbf{v}_2 & \cdots & \mathbf{v}_n \end{bmatrix} =
\begin{bmatrix} \mathbf{v}_1 \cdot \mathbf{v}_1 & \mathbf{v}_1 \cdot \mathbf{v}_2 & \cdots & \mathbf{v}_1 \cdot \mathbf{v}_n \\
\mathbf{v}_2 \cdot \mathbf{v}_1 & \mathbf{v}_2 \cdot \mathbf{v}_2 & \cdots & \mathbf{v}_2 \cdot \mathbf{v}_n \\ \vdots & \vdots & \ddots & \vdots
\\ \mathbf{v}_n \cdot \mathbf{v}_1 & \mathbf{v}_n \cdot \mathbf{v}_2 & \cdots & \mathbf{v}_n \cdot \mathbf{v}_n \end{bmatrix}$$

## We use the function **cublasSgemm** to perform the above matrix multiplication.  
## Here **Sgemm** stands for single precision general matrix multiply.  
## This **cuBLAS** function allows us to take full advantage of the awesome number crunching capabilities of the GPU (including Tensor Cores!) without writing complicated CUDA kernels.
## Note that to use **cuBLAS** our matrices have to be stored in **column major order**.  
## Fortunately, we learned previously that it is typically much faster to work with matrices in **column major order** in CUDA because it allows **warps of threads** to access **consecutive memory locations**.  
## To avoid using up too much memory we separate computing the large matrix product into pieces.

## The full details of how everything works together to achieve high performance in the CUDA code below is beyond the scope of this lecture.  

## **If you are interested in learning more about programming for high performance in CUDA you should take CMDA 4634 (Scalable Computing for CMDA).**  

# **Switch runtime to a T4**

In [1]:
!git clone https://code.vt.edu/jasonwil/cmda3634_materials.git

Cloning into 'cmda3634_materials'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 282 (delta 89), reused 9 (delta 2), pack-reused 37 (from 1)
Receiving objects: 100% (282/282), 43.40 MiB | 20.64 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [2]:
# copy the lecture 18 files to our working directory
!cp cmda3634_materials/L18/* .
# uncompress the .gz files
!gzip -d *.gz

In [3]:
%%writefile gpu_extreme_v5.cu
#include <stdio.h>
#include <stdlib.h>
#include <float.h>
#include <time.h>
#include <cuda.h>
#include <cublas_v2.h>

typedef unsigned char byte;
typedef unsigned long long uint64;

// read data from a binary file
void read_bin (byte* data, int num_bytes, char* filename, int header_size) {
    byte header[header_size];
    FILE* fptr;
    int num_read;
    // open the binary file for reading
    fptr = fopen(filename,"rb");
    // need to check for null
    if (fptr == 0) {
        printf ("Error opening binary data file %s.\n",filename);
        exit(1);
    }
    // read header
    num_read = fread(header, sizeof(byte), header_size, fptr);
    // read data
    num_read = fread(data, sizeof(byte), num_bytes, fptr);
    if (num_read != num_bytes) {
        printf ("Warning : binary data file read error for %s.\n",filename);
    }
    // close the binary file
    fclose(fptr);
}

void extreme_info_expand(uint64 info, int* dist_sq, int* i, int* j) {
    *j = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *i = info & 0xFFFF; // apply bit mask to zero out all but lower 16 bits
    info = info >> 16; // shift 16 bits to the right
    *dist_sq = info;
}

__device__ uint64 extreme_info_compress(int dist_sq, int i, int j) {
    uint64 info = dist_sq;
    info = info << 16; // shift 16 bits to the left
    info += i;
    info = info << 16; // shift 16 bits to the left
    info += j;
    return info;
}

__global__ void extremeKernel(float* dot_prods, float* i_j_dot_prods, int len,
			      int start, int size, uint64* max_info) {

    int t = blockIdx.x*blockDim.x + threadIdx.x;
    if (t < size) {
	    float term1 = dot_prods[t+start];
        float max_dist_sq = 0;
        int farthest_idx;
	    for (int j=0;j<t+start;j++) {
            float term2 = term1 + dot_prods[j];
            float term3 = i_j_dot_prods[j*size+t];
            float dist_sq = term2 - 2.0*term3;
            if (dist_sq > max_dist_sq) {
                max_dist_sq = dist_sq;
                farthest_idx = j;
            }
        }
	    uint64 info = extreme_info_compress((int)max_dist_sq,t+start,farthest_idx);
	    atomicMax(max_info,info);
    }
}

__global__ void dotprodsKernel (float *data, int len, int dim, float* dot_prods) {
    int t = blockIdx.x*blockDim.x + threadIdx.x;
    if (t < len) {
        float result = 0;
        for (int i=0;i<dim;i++) {
            float term = data[i*len+t];
            result += term*term;
        }
        dot_prods[t] = result;
    }
}

int main (int argc, char** argv) {

    // read in a MNIST image set
    int len = 60000;
    int dim = 784;
    byte* data_bytes = (byte*)malloc(len*dim*sizeof(byte));
    // read the file that thas the dataset stored in column major order
    char images_file[] = "train-images-idx3-ubyte-c";
    read_bin(data_bytes,len*dim,images_file,16);

    // translate input data from byte to float matrices
    // so that we can use CUBLAS
    float* data = (float*)malloc(len*dim*sizeof(float));
    for (int i=0;i<len*dim;i++) {
        data[i] = data_bytes[i];
    }

    // allocate device memory
    int size = 30000;
    float* d_data;
    uint64* d_max_info;
    float* d_dot_prods;
    float* d_i_j_dot_prods;
    cudaMalloc(&d_data,len*dim*sizeof(float));
    if (d_data == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_max_info,sizeof(uint64));
    if (d_max_info == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_dot_prods,len*sizeof(float));
    if (d_dot_prods == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }
    cudaMalloc(&d_i_j_dot_prods,(long)size*len*sizeof(float));
    if (d_i_j_dot_prods == NULL) {
	    printf ("cudaMalloc failed\n");
	    return 1;
    }

    // start the timer
    clock_t start = clock();

    // copy data to device
    cudaMemcpy(d_data,data,len*dim*sizeof(float),cudaMemcpyHostToDevice);

    // initialize the device max_info to 0
    cudaMemset(d_max_info,0,sizeof(uint64));

    // launch kernel to compute i-i dot products
    int B = 128;
    int G = (len+B-1)/B;
    dotprodsKernel <<< G, B >>> (d_data,len,dim,d_dot_prods);

    // setup CUBLAS
    float alpha = 1.0, beta = 0;
    cublasHandle_t handle;
    cublasCreate(&handle);
    // this next command enables the Tensor Cores!
    cublasSetMathMode(handle, CUBLAS_TENSOR_OP_MATH);

    G = (size+B-1)/B;
    for (int start = 0; start < len; start += size) {

	    // use CUBLAS to compute a batch of i-j dot products
	    cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_T,
		    size, len, dim, &alpha,
		    d_data+start, len,
		    d_data, len, &beta,
		    d_i_j_dot_prods, size);
	    cudaDeviceSynchronize();

	    // launch kernel to compute a batch of extreme distances
	    extremeKernel <<< G, B >>> (d_dot_prods,d_i_j_dot_prods,len,start,size,d_max_info);
    }

    // copy max_info from device to host
    uint64 max_info;
    cudaMemcpy(&max_info,d_max_info,sizeof(uint64),cudaMemcpyDeviceToHost);

    // stop the timer
    clock_t stop = clock();
    double elapsed = (double)(stop-start)/CLOCKS_PER_SEC;

    // expand max_info
    int max_dist_sq, i, j;
    extreme_info_expand(max_info,&max_dist_sq,&i,&j);

    // print results
    printf ("number of images = %d\n",len);
    printf ("elapsed time = %.4f seconds\n",elapsed);
    printf ("extreme distance = %.2f\n",sqrt(max_dist_sq));
    printf ("extreme pair = (%d,%d)\n",i,j);

    // free dynamically allocated memory
    free(data_bytes);
    free(data);
    cudaFree(d_data);
    cudaFree(d_max_info);
    cudaFree(d_dot_prods);
    cudaFree(d_i_j_dot_prods);
}

Writing gpu_extreme_v5.cu


In [4]:
!nvcc -arch=sm_75 -o gpu_extreme_v5 gpu_extreme_v5.cu -lcublas

In [5]:
!./gpu_extreme_v5

number of images = 60000
elapsed time = 0.6152 seconds
extreme distance = 4303.32
extreme pair = (59452,26785)


# **Switch runtime to a CPU**

## Recall that we have access to Nvidia A100 GPUs on ARC.
## The A100 GPU is extremely fast at matrix multiplication.  
## Here is a run on an A100 where we calculate the extreme pair of 60000 images.

    [jasonwil@tc-dgx010]$ nvcc -arch=sm_80 -o gpu_extreme_v5 gpu_extreme_v5.cu -lcublas
    [jasonwil@tc-dgx010]$ ./gpu_extreme_v5
    number of images = 60000
    elapsed time = 0.1015 seconds
    extreme distance = 4303.32
    extreme pair = (59452,26785)

## Recall that for the seqential code compiled with the optimization **-O3** turned on we calculated:

    number of images = 60000
    elapsed time = 209.5897 seconds
    extreme distance = 4303.32
    extreme pair = (26785,59452)

## Note that speedup for 60000 images is
## $$\text{speedup} = \displaystyle\frac{\text{Sequential Time}}{\text{A100 GPU Time}} = \displaystyle\frac{209.5897}{0.1015} = 2065$$
## **The A100 GPU is around 2065 times faster than the CPU for this problem!**